# Segun video
## Usamos chroma mediante Langchain
Todo el Notebook se basa en este video: https://www.youtube.com/watch?v=eHfMCtlsb1o

In [27]:
import ollama
from pydantic import BaseModel
from typing import List
import pickle as pkl

Primero tenemos que leer el fichero badbunny. Se encuenta en `understanding/structured_data_extraction/bad_bunny.txt` y este Notebook en `understanding/structured_data_extraction/ollama_json.ipynb`

In [47]:
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnablePassthrough

# Load the text
loader = TextLoader("bad_bunny.txt")
docs = loader.load()

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

Vemos la estructura de splits.

In [8]:
splits

[Document(page_content='Benito Antonio Martínez Ocasio (Vega Baja, Puerto Rico 10 de marzo de 1994),2\u200b conocido artísticamente como Bad Bunny, es un cantante, compositor, productor musical y peleador aficionado de lucha libre puertorriqueño.3\u200b Sus estilos musicales son generalmente definidos como reguetón, trap latino y baladas, aunque también ha interpretado otros géneros y estilos variados. Se caracteriza por su entonación grave y su estilo de vestir.4\u200b', metadata={'source': 'bad_bunny.txt'}),
 Document(page_content='Empezó a ganar popularidad en SoundCloud y finalmente, firmó un contrato con el sello discográfico Hear This Music mientras trabajaba como empaquetador en un supermercado y estudiaba en la Universidad de Puerto Rico en Arecibo. Después del éxito de su sencillo «Soy peor» en 2016, alcanzó la fama tras colaborar con los artistas Cardi B y Drake en los sencillos «I Like It» y «Mia» que alcanzaron el primer y tercer puesto en la lista Billboard Hot 100 respect

In [48]:
# Create Ollama embeddings and vector store
embedding_model = OllamaEmbeddings(model="llama2")

Codigo para comprobar/entender que cada cacho de documento de split esta asociado a una lista vectores de vectorstore

In [49]:
persist_directory = "embeddings_langchain"
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory=persist_directory)

Definimos,

- **retriver** Se encarga de calcular la similaridad con los documentos de la base de datos Chroma
- `rag_chain` usa retriever para encontrar los documentos mas similares al input (donde, probablemente, estara la respuesta a la pregunta)
- `format_docs` junta esos documentos en los que puede estar la respuesta y los junta en una string.
- `ollama_lm` es la llamada al modelo llama2 junto con todos los documentos en los que seguramente este la respuesta para que de una respuesta.

In [54]:
# Create the retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama2', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    for doc in retrieved_docs:
        if 'spotify' in doc.page_content.lower():
            print(doc.page_content)
            print("\n\n")
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

# Use the RAG chain
result = rag_chain("Entre que años fue Bad Bunny el artista más escuchado en Spotify?")
print(result)

Bad Bunny fue nominado a los Premios Tu Música Urbano inaugurales de Telemundo en 2019 en la categoría de "Premio humanitario del Año". Además, ha participado en various events and protests related to social and political issues in Puerto Rico, including joining other artists to shut down a highway in protest of a governor's comments that were deemed offensive to the LGBTQ+ community.

In terms of his music career, Bad Bunny is primarily known as a reggaeton and trap artist, and has been featured on the cover of Rolling Stone magazine as well as other prominent publications. He has also made appearances in the WWE and has performed his song "Booker T" live at the Royal Rumble event in 2021.

Therefore, the answer to the question is between 2019 and 2021, as that is when Bad Bunny was nominated for a prize and began his career in the WWE.


No esta extrayendo el documento correct, ha acertado de chiripa

La respuesta esta en esta frase de wikipedia (de donde he sacado el txt)  https://es.wikipedia.org/wiki/Bad_Bunny: 
*"Además, fue el primer artista hispanohablante en ser el más reproducido en Spotify entre 2020 y 2022"*

Aqui podemos ver que sin context el modelo no responde bien. Escribi esto antes de ver el resultado. Por eso antes aunque no ha sacado bien el documento ha contestado bien.

In [52]:
ollama.chat(model='llama2', messages=[{'role': 'user', 'content': "Entre que años fue Bad Bunny el artista más escuchado en Spotify?"}])

{'model': 'llama2',
 'created_at': '2024-04-18T16:49:00.291329Z',
 'message': {'role': 'assistant',
  'content': 'Bad Bunny se convirtió en el artista más escuchado en Spotify en 2019. Según datos de Spotify, en ese año Bad Bunny fue el artista más escuchado en la plataforma, con más de 300 millones de horas de reproducción mensuales.'},
 'done': True,
 'total_duration': 14661961000,
 'load_duration': 12673708,
 'prompt_eval_count': 25,
 'prompt_eval_duration': 7913164000,
 'eval_count': 71,
 'eval_duration': 6719005000}

## Contestar con JSON


Ahora queremos que responda con un JSON

https://www.youtube.com/watch?v=eHfMCtlsb1o

In [58]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor

Esta clase de pydanctic comprueba que el json que devuelva sea valido (sobre todo es util para el tipo de dato).

In [59]:
class BadBunnyInfo(BaseModel):
    artist_name: str = Field(..., description="Artist name of the singer")
    name: str = Field(..., description="Real Name")
    surname: str = Field(..., description="Real Surname")
    birth_date: str = Field(..., description="Birth Date")
    year_YHLQMDLG: int = Field(..., description="Year of the album YHLQMDLG")

Llamamos a la api con el modelo llama2 como en el video

In [61]:
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)


Todo lo que esta comentado es porque no funcionaba. Voy a definir un texto muy sencillo.

In [63]:
# queries = [
# "Nombre artistico del cantante de Bad Bunny",
# "Como se llama el cantante Bad Bunny",
# "Apellidos del cantante Bad Bunny",
# "Fecha de Nacimiento del cantante Bad Bunny",
# "En que año lanzo su album YHLQMDLG",
# ]

In [64]:
# retrieved_docs = {}
# for query in queries:
#     retrieved_doc = retriever.invoke(query)
#     retrieved_docs[query] = retrieved_doc

In [65]:
# result = ""

# for query, doc in retrieved_docs.items():
#     print(query)
#     result += query
#     print("\n")
#     result += "\n"
#     for page in doc:
#         print(page.page_content)
#         result += page.page_content
#         print("\n")
#         result += "\n"

In [66]:
# retrieved_docs['Fecha de Nacimiento del cantante Bad Bunny']

Si no es capaz de sacar la informacion de este texto, yo ya me jubilo.

In [67]:
# content = """
# Hola me llamo Bad Bunny, mi nombre real es Benito Antonio Martínez Ocasio, nací el 10 de marzo de 1994 y mi album YHLQMDLG lo publiqué en 2020.
# """

# Write same content but in english
content = """
Please, write the the artist name and birth date mentioned in this text: Hello my name as singer is Bad Bunny, my real name is Benito Antonio Martínez Ocasio, I was born on March 10, 1994 and I released my album YHLQMDLG in 2020.
"""
# content = """
# Tienes en un texto la información de un cantante. El texto contiene información sobre su nombre real, su fecha de nacimiento y el año en el que su álbum YHLQMDLG. Aqui puedes encontrar el texto: 
# {texto}

# Por favor, responde a las siguientes preguntas:
# 1. Cual es su nombre artistico
# 2. Cual es su nombre real
# 3. Cual es su apellido
# 4. Cual es su fecha de nacimiento
# 5. En que año publico su album YHLQMDLG

# Responde con un JSON con keys
# 1. 'artirt_name'
# 2. 'name'
# 3. 'surname'
# 4. 'birth_date'
# 5. 'year_YHLQMDLG'
# """

Defino el pydantic en ingles a ver si asi funciona

In [68]:
class BadBunnyInfo(BaseModel):
    artist_name: str = Field(..., description="Artist name of the singer")
    #name: str = Field(..., description="Real Name")
    #surname: str = Field(..., description="Real Surname")
    #birth_date: str = Field(..., description="Birth Date")
    year_YHLQMDLG: int = Field(..., description="Year of the album YHLQMDLG")

In [71]:
resp = client.chat.completions.create(
    model="llama2",
    messages=[
        {
            "role": "user",
            #"content": content.format(texto=result)
            "content": content
        }
    ],
    response_model=BadBunnyInfo,
    max_retries=10
)

In [72]:
resp

BadBunnyInfo(artist_name='Bad Bunny', year_YHLQMDLG=2020)

Funciona :))))))))))))))))))))

Lo probamos con el texto grande

In [73]:
queries = [
    "Nombre artistico del cantante de Bad Bunny",
    "Como se llama el cantante Bad Bunny",
    "Apellidos del cantante Bad Bunny",
    "Fecha de Nacimiento del cantante Bad Bunny",
    "En que año lanzo su album YHLQMDLG",
]

Otro ejemplo

In [75]:
class Currency(BaseModel):
    symbol: str = Field(..., description="ISO 4217 currency code")

In [76]:
resp = client.chat.completions.create(
    model="llama2",
    messages=[
        {
            "role": "user",
            #"content": content.format(texto=result)
            "content": "What is the currency code for the US Dollar?"
        }
    ],
    response_model=Currency,
    max_retries=10
)
resp

Currency(symbol='USD')

# Mi manera con ChromaDB

In [77]:
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Load the text
loader = TextLoader("bad_bunny.txt")
docs = loader.load()

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

Creamos una funcion en la que metemos texto y nos devuelve el vector del embedding generado por llama2.  Esto SOLO lo pasa a vectores. NO TOKENIZA. Es decir, cada documento tendra asociado un vector.

In [78]:
# Generamos los embeddings de cada chunk con una funcion para no generar los embeddings sin querer
def generate_embeddings(prompt: str = 'The sky is blue because of rayleigh scattering'):
    vector = ollama.embeddings(model='llama2', prompt=prompt)
    return vector

In [79]:
example = generate_embeddings()
print(type(example))
print(example)
print(f"Ollama2 transforms the words in vectors of dimension: {len(example['embedding'])}")

<class 'dict'>
{'embedding': [0.8026409149169922, -0.6997383832931519, 0.6685507297515869, 2.087705373764038, -0.3597745895385742, -0.7193409204483032, -0.3752881586551666, 0.7305898666381836, 0.220156729221344, -0.6775131821632385, 0.3040643036365509, -1.3673839569091797, -0.13077086210250854, 0.11252722144126892, 0.5427549481391907, -0.5427400469779968, -1.302517056465149, 1.1957448720932007, 0.9690303206443787, 1.6445399522781372, 1.8558064699172974, -3.7210965156555176, 0.659212589263916, -3.0161290168762207, 0.16594401001930237, -2.4745938777923584, -0.6295669674873352, 1.6412791013717651, 0.2634392976760864, 1.830955147743225, -0.6450852155685425, -1.1743316650390625, 0.2681480348110199, 2.0101146697998047, -0.25673824548721313, -0.5765746831893921, 1.0189534425735474, -1.434535264968872, 0.8933296203613281, -2.3046984672546387, 0.6063656210899353, -3.1889917850494385, 2.5508172512054443, 0.2089187204837799, -3.47857666015625, 1.5960805416107178, -1.0160739421844482, 1.3754639625

## Conseguir vectorizar todos los documentos

In [80]:
import pickle as pkl
vectors = []
for split in splits:
    vector_dict = generate_embeddings(split.page_content)
    vectors.append(vector_dict['embedding'])

# Guardamos los embeddings en un archivo
with open('vectors_badbunny.pkl', 'wb') as f:
    pkl.dump(vectors, f)

In [81]:
len(vectors)

46

persist_directory es el directorio donde vamos a guardar la base de datos de los vectores.

Cada documento en split esta asociado a una lista de vectores en vectorstore.

In [82]:
# Creamos el cliente de la Base de Datos y le asociamos un path para guardar los embeddings
import chromadb
client = chromadb.Client()
client = chromadb.PersistentClient(path="embeddings")